In [1]:
# Imports
import random
import sys
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import torch.nn as nn
import torch.optim as optim
import torchvision.utils as vutils
import matplotlib.animation as animation
from IPython.display import HTML
from torchsummary import summary

import model_v4_small as model
#import model

import torch

import keijzer_exogan as ke

# initialize random seeds
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)

%matplotlib inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"} # Make sure the axis background of plots is white, this is usefull for the black theme in JupyterLab
# Initialize default seaborn layout
sns.set_palette(sns.hls_palette(8, l=.3, s=.8))
sns.set(style='ticks')


"""
Local variables
"""
workers = 0 # Number of workers for dataloader, 0 when to_vram is enabled
batch_size = 1 # using one image ofcourse
image_size = 32
nz = 100 # size of latent vector
n_iters = 1000 #25*10**3 # number of iterations to do for inpainting
torch.backends.cudnn.benchmark=True # Uses udnn auto-tuner to find the best algorithm to use for your hardware, speeds up training by almost 50%

lr = 1e-1
lamb1 = 1 #1e4
lamb2 = 1e-1 # 1 , total_loss = lamb1*loss_context + lamb2*loss_perceptual

beta1 = 0.5 # Beta1 hyperparam for Adam optimizers
selected_gpus = [0] # Number of GPUs available. Use 0 for CPU mode.

#n_images = 500
inpaint_n_times = 1

save_array_results = False
load_array_results = False
filename = 'debug_0_100_1_1e-1_gan' # 0:100 lamb1=10, lamb2=1

ngpu = len(selected_gpus)

In [2]:
"""
Load and setup models
"""
# Initialize cuda
device = torch.device("cuda:"+str(selected_gpus[0]) if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Load models, set to evaluation mode since training is not needed (this also allows batchsize 1 to work with batchnorm2d layers)
netG = model.Generator(ngpu).eval().to(device)
netD = model.Discriminator(ngpu).eval().to(device)

# Apply weights
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

netG.apply(weights_init) # It's not clean/efficient to load these ones first, but it works.
netD.apply(weights_init)





"""
Define input training stuff (fancy this up)
"""
G = netG
D = netD
z = torch.randn(1, nz, 1, 1, requires_grad=True, device=device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    G = nn.DataParallel(G, device_ids=selected_gpus, output_device=device)
    D = nn.DataParallel(D, device_ids=selected_gpus, output_device=device)
    #z = nn.DataParallel(z, device_ids=selected_gpus, output_device=device)

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999)) # should be sgd
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

print('done')

done


In [3]:
z.shape

torch.Size([1, 100, 1, 1])

In [4]:
summary(G, input_size=(100, 1, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 128, 2, 2]          51,200
         LeakyReLU-2            [-1, 128, 2, 2]               0
   ConvTranspose2d-3             [-1, 64, 4, 4]         131,072
         LeakyReLU-4             [-1, 64, 4, 4]               0
   ConvTranspose2d-5             [-1, 32, 8, 8]          32,768
         LeakyReLU-6             [-1, 32, 8, 8]               0
   ConvTranspose2d-7           [-1, 32, 16, 16]          16,384
         LeakyReLU-8           [-1, 32, 16, 16]               0
   ConvTranspose2d-9            [-1, 1, 32, 32]             512
             Tanh-10            [-1, 1, 32, 32]               0
Total params: 231,936
Trainable params: 231,936
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.20
Params size (MB): 0.88
Estimated T

In [5]:
summary(D, input_size=(1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]             512
         LeakyReLU-2           [-1, 32, 16, 16]               0
            Conv2d-3             [-1, 64, 8, 8]          32,768
         LeakyReLU-4             [-1, 64, 8, 8]               0
            Conv2d-5            [-1, 128, 4, 4]         131,072
         LeakyReLU-6            [-1, 128, 4, 4]               0
            Conv2d-7            [-1, 256, 2, 2]         524,288
         LeakyReLU-8            [-1, 256, 2, 2]               0
            Conv2d-9              [-1, 1, 1, 1]           1,024
          Sigmoid-10              [-1, 1, 1, 1]               0
Total params: 689,664
Trainable params: 689,664
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.23
Params size (MB): 2.63
Estimated T